In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import mglearn  # Generates a deprecation warning

/usr/local/lib64/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/usr/local/lib64/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Data import

We start by importing our processed data, dropping columns unhelpful for classification, and pulling out the target column from the rest of the observations.

In [2]:
dnd = pd.read_csv("dnd_monsters_processed.csv")
dnd = dnd.drop(columns=["Index", "name", "size", "type"])
X = dnd.drop(columns=["cr"]).to_numpy()
y = dnd.cr.to_numpy()

Since not all models can take real-numbered targets, we also create a shifted target vector that converts the fractional challenge ratings the lowest-valued integers and shift the rest of the challenge ratings to accommodate them.

In [3]:
y_shift = []
for i in y:
    if i == .125:
        y_shift.append(1)
    elif i == .25:
        y_shift.append(2)
    elif i == .5:
        y_shift.append(3)
    else:
        y_shift.append(i+3)

# Testing Methods

To run our experiments, we created a helper function that takes a model, the observations, target values, and a number of trials to run. This allowed us to easily run experiments per model and find an average score for them.

In [4]:
def test_model(model, X, y, trials):
    test = []
    train = []
    for i in range(trials):
        X_train, X_test, y_train, y_test = train_test_split(X, y)    
        model.fit(X_train, y_train)
        test.append(model.score(X_test, y_test))
        train.append(model.score(X_train, y_train))

    print("Accuracy on training set: {:.5f}".format(np.mean(train)))
    print("Accuracy on test set: {:.5f}".format(np.mean(test)))

# KNN Classification and Regression

As a baseline, we first started by trying to use knn classification and regression schemes. Since knn models cannot use continuous target values, we had to use the shifted targets. Starting with knn classification, we see that it wasn't very effective.

In [5]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(15)
test_model(knn, X, y_shift, 100)

Accuracy on training set: 0.39266
Accuracy on test set: 0.27702


But when using the knn regression model, we find that it works much better.

In [6]:
from sklearn.neighbors import KNeighborsRegressor
knn_reg = KNeighborsRegressor(15)
test_model(knn_reg, X, y_shift, 100)

Accuracy on training set: 0.91836
Accuracy on test set: 0.90675


# Linear Regression

Seeing that regression worked better than classification, we continue down the regression path and use the linear regression model. This ran much faster compared to knn regression, but had a slight hit to accuracy.

In [7]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
test_model(lr, X, y, 100)

Accuracy on training set: 0.90444
Accuracy on test set: 0.89304


# MLP Regression

We then used a multilevel perception model. This was the slowest to run of them all, so we had to reduce the number of trials to get results in a reasonable amount of time, but it had the most accuracy overall.

In [8]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor()
test_model(mlp, X, y, 10)

/usr/local/lib64/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib64/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib64/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib64/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarni

Accuracy on training set: 0.94673
Accuracy on test set: 0.91905


/usr/local/lib64/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
